In [2]:
import os  # For process ID
import requests
import random
import time
import numpy as np
import psycopg2


def quantum_random_number(min_val, max_val, count=1):
    """
    Generates random numbers using the QRNG API.
    Falls back to classical randomness if the QRNG API fails.
    """
    try:
        # ANU QRNG API URL
        api_url = "https://qrng.anu.edu.au/API/jsonI.php"
        response = requests.get(api_url, params={"length": count, "type": "uint8"})
        response.raise_for_status()  # Raise an error for HTTP issues
        
        data = response.json()
        if not data["success"]:
            raise RuntimeError("QRNG API returned an unsuccessful response.")
        
        # Map the random numbers to the desired range
        random_numbers = [min_val + (num % (max_val - min_val + 1)) for num in data["data"]]
        return random_numbers, 'quantum'
    except Exception:
        # Fall back to classical randomness if QRNG API fails
        random_numbers = [random.randint(min_val, max_val) for _ in range(count)]
        return random_numbers, 'classical'


def generate_powerball_ticket():
    """
    Generates a single Powerball ticket.
    Returns a tuple containing the ticket and the randomness source.
    """
    white_balls = []
    source = None
    while len(white_balls) < 5:
        number, source = quantum_random_number(1, 69, count=1)
        if number[0] not in white_balls:
            white_balls.append(number[0])
    white_balls.sort()

    # Generate the Powerball
    powerball, source = quantum_random_number(1, 26, count=1)
    return (white_balls, powerball[0], source)


def generate_embedding(white_balls, powerball):
    """
    Creates a vector embedding for a Powerball ticket.
    Normalizes values to range [0, 1].
    """
    normalized_white_balls = [num / 69.0 for num in white_balls]
    normalized_powerball = powerball / 26.0
    embedding = np.array(normalized_white_balls + [normalized_powerball])
    return embedding


def insert_ticket_into_db(white_balls, powerball, source, process_id, embedding):
    """
    Inserts a Powerball ticket with vector embedding into the PostgreSQL database.
    """
    try:
        conn = psycopg2.connect(
            dbname="rag_db",      # Replace with your database name
            user="user",          # Replace with your database username
            password="password",  # Replace with your database password
            host="192.168.0.34",  # Replace with your database host
            port=5432             # Replace with your database port
        )
        cursor = conn.cursor()

        # Insert query
        insert_query = """
        INSERT INTO powerball_ticket_sessions (white_ball_1, white_ball_2, white_ball_3, white_ball_4, white_ball_5, powerball, source, process_id, embedding)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s);
        """
        cursor.execute(insert_query, (*white_balls, powerball, source, process_id, embedding.tolist()))
        conn.commit()
        print("Ticket successfully inserted into the database with embedding.")
    except Exception as e:
        print(f"Database error: {e}")
    finally:
        cursor.close()
        conn.close()


def continuously_generate_powerball():
    """
    Continuously generates Powerball tickets, stores them in a database,
    and handles interruptions or errors.
    """
    process_id = os.getpid()  # Get the process ID
    winning_ticket = None
    while True:
        try:
            # Generate a Powerball ticket
            winning_ticket = generate_powerball_ticket()
            white_balls, powerball, source = winning_ticket

            # Generate the embedding
            embedding = generate_embedding(white_balls, powerball)

            # Print the ticket
            print(f"Generated Ticket: White Balls: {white_balls}, Powerball: {powerball} (Source: {source}, Process ID: {process_id})")

            # Insert the ticket into the database
            insert_ticket_into_db(white_balls, powerball, source, process_id, embedding)

            time.sleep(1)  # Avoid excessive API requests
        except KeyboardInterrupt:
            print("\nProcess interrupted by the user.")
            if winning_ticket:
                white_balls, powerball, source = winning_ticket
                print(f"The WINNING TICKET is: White Balls: {white_balls}, Powerball: {powerball} (Source: {source}, Process ID: {process_id})")
            else:
                print("No ticket was generated before the interruption.")
            break
        except Exception as e:
            print(f"\nInternet connection lost! Retrying... Error: {e}")
            time.sleep(5)  # Retry after a delay


if __name__ == "__main__":
    continuously_generate_powerball()


Generated Ticket: White Balls: [15, 45, 50, 62, 68], Powerball: 14 (Source: classical, Process ID: 2962481)
Ticket successfully inserted into the database with embedding.
Generated Ticket: White Balls: [2, 37, 41, 51, 60], Powerball: 21 (Source: classical, Process ID: 2962481)
Ticket successfully inserted into the database with embedding.
Generated Ticket: White Balls: [2, 29, 30, 45, 67], Powerball: 8 (Source: classical, Process ID: 2962481)
Ticket successfully inserted into the database with embedding.
Generated Ticket: White Balls: [1, 3, 27, 33, 67], Powerball: 26 (Source: classical, Process ID: 2962481)
Ticket successfully inserted into the database with embedding.
Generated Ticket: White Balls: [13, 29, 37, 58, 63], Powerball: 6 (Source: classical, Process ID: 2962481)
Ticket successfully inserted into the database with embedding.
Generated Ticket: White Balls: [5, 6, 20, 24, 67], Powerball: 18 (Source: classical, Process ID: 2962481)
Ticket successfully inserted into the databas